Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 00_Environment_Setup

_**Setting up your Azure Machine Learning services workspace for this solution accelerator**_

---

## 1.0 Verify Azure ML SDK Version

In [ ]:
import azureml.core

print("This notebook was created using version 1.0.74.1 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

If you are using an older version of the SDK then this notebook was created using, you should upgrade your SDK.

## 2.0 Register AKS provider
Azure Machine Learning uses of [Azure Container Instance (ACI)](https://azure.microsoft.com/services/container-instances) to deploy dev/test web services. An Azure subscription needs to be registered to use ACI.  If you or the subscription owner have not yet registered ACI on your subscription, you will need to use the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) and execute the following commands.

```shell
# check to see if ACI is already registered
(myenv) $ az provider show -n Microsoft.ContainerInstance -o table

# if ACI is not registered, run this command.
# note you need to be the subscription owner in order to execute this command successfully.
(myenv) $ az provider register -n Microsoft.ContainerInstance
```


## 3.0 Connect to your Azure ML workspace

### Workspace parameters

To configure this solution accelerator to use this AML Workspace, supply the following information:
* Your subscription id
* A resource group name
* (optional) The region that will host your workspace
* A name for your workspace

The following cell allows you to specify your workspace parameters.  This cell uses the python method `os.getenv` to read values from environment variables which is useful for automation.  If no environment variable exists, the parameters will be set to the specified default values.  

Replace the default values in the cell below with your workspace parameters:

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="")
resource_group = os.getenv("RESOURCE_GROUP", default="")
workspace_name = os.getenv("WORKSPACE_NAME", default="")
workspace_region = os.getenv("WORKSPACE_REGION", default="")

## 4.0 Access workspace & Write config

The following cell uses the Azure ML SDK to attempt to load the workspace specified by your parameters.  If this cell succeeds, your notebook library will be configured to access the workspace from all notebooks using the `Workspace.from_config()` method.  The cell can fail if the specified workspace doesn't exist or you don't have permissions to access it. 

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

## 5.0 Create a compute cluster

The following cell will create a Aml compute cluster to run our training, scoring, and forecasting pipelines. This is a one-time set up that you don't need to re-run.

We created a Amlcompute cluster called 'train-many-model'. The VM size is STANDARD_D13_V2 with a minimal node of 5.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "train-many-model"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=5,
                                                           max_nodes=20)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## 6.0 Register the blob containers to the Workspace

We created 3 separate blob containers to store our training, scoring, and forecasting outputs. You will need to register the 3 containers to the Workspace so that you can call the corresponding datastore in its pipeline notebook. This is a one-time set up that you don't need to re-run.

We will need to find the blob account name, and account key first. In the Azure portal, find your storage account. Under the 'Settings' click the 'Access keys' tab. You should be able to find the 'Storage account name' and 'Key'. Fill the values into the next code block.

In [ ]:
account_name = "<my-account-name>" # Storage account name
account_key = "<my-account-key>" # Storage account key

Then we can register the blob containers to the Workspace. Our 3 blob containers names are 'training-output', 'scoring-output', and 'forecasting-output'.

- **datastore_name** is the registered datastore name that we call later in our notebooks and scripts.

In [ ]:
from azureml.core import Datastore

training_output_datastore = Datastore.register_azure_blob_container(
           workspace=ws,
           datastore_name='training_output_datastore', # Registered datastore name
           account_name=account_name,
           container_name='training-output', # Name of Azure blob container
           account_key=account_key)

In [ ]:
scoring_output_datastore = Datastore.register_azure_blob_container(
           workspace=ws,
           datastore_name='scoring_output_datastore',
           account_name=account_name, # Storage account name
           container_name='scoring-output', # Name of Azure blob container
           account_key=account_key # Storage account key)

In [ ]:
forecasting_output_datastore = Datastore.register_azure_blob_container(
           workspace=ws,
           datastore_name='forecasting_output_datastore',
           account_name=account_name, # Storage account name
           container_name='forecasting-output', # Name of Azure blob container
           account_key=account_key # Storage account key)

---

## Next steps

Now that you've created your configuration file, you're all set to move on to the next step. Move on to the data preparation notebook in the next folder.